In [134]:
import pandas as pd
import numpy as np

In [165]:
class ValidityCheck:
    # Battery Specifications
    mlf = 0.991                 # Marginal Loss Factor
    battery_capacity = 580      # Battery Capacity
    battery_power = 300         # Battery Power
    charge_efficiency = 0.9     # Charge Efficiency
    discharge_efficiency = 0.9  # Discharge Efficiency
    
    def __init__(self, source, result, training_period = None, 
                 cv_period = None, testing_period = None, region = "VIC"):
        self.og_df = source   # Original DataFrame
        self.region = region  # Region of the resulting DataFrame
        
        self.train_period = training_period
        self.cv_period = cv_period
        self.test_period = testing_period
        
        if self.region == "VIC":
            self.price_name = "Regions VIC Trading Price ($/MWh)"
        elif self.region == "NSW":  
            self.price_name = "Regions NSW Trading Price ($/MWh)"
        elif self.region == "SA":
            self.price_name = "Regions SA Trading Price ($/MWh)"
        elif self.region == "TAS":
            self.price_name = "Regions TAS Trading Price ($/MWh)"
        
        self.re_df = self.rename_columns(result)   # DataFrame that wants to be checked
        self.validation()
        
        self.check_price_period()
        #if (self.check_price()):
        #    self.check()
        #else:
        #    print("Submitted File has different price format!")
                    
    def rename_columns(self, dataframe):
        copy_df = dataframe.copy()
        
        copy_df = copy_df.rename(columns = {"Price": self.price_name})
        
        copy_df = copy_df.rename(columns = {"Time": "Time (UTC+10)"})
        copy_df = copy_df.rename(columns = {"Dispatch": "Market Dispatch (MWh)"})
        copy_df = copy_df.rename(columns = {"Open": "Opening Capacity (MWh)"})
        copy_df = copy_df.rename(columns = {"Close": "Closing Capacity (MWh)"})
        
        return copy_df
    
    def check_price_period(self):
        mismatch = []
        nonconti = []
        self.og_df["Time (UTC+10)"] = pd.to_datetime(self.og_df['Time (UTC+10)'])
        self.re_df["Time (UTC+10)"] = pd.to_datetime(self.re_df['Time (UTC+10)'])
        if len(self.og_df) == len(self.re_df):
            for i in range(1, len(self.re_df)):
                curr_og = self.og_df.loc[i-1, "Time (UTC+10)"]
                curr_re = self.re_df.loc[i-1, "Time (UTC+10)"]
                
                curr_og_price = self.og_df.loc[i-1, self.price_name]
                curr_re_price = self.re_df.loc[i-1, self.price_name]
                
                next_og = self.og_df.loc[i, "Time (UTC+10)"]
                next_re = self.re_df.loc[i, "Time (UTC+10)"]
                
                re_diff = (next_re - curr_re).seconds / 60
                if curr_og != curr_re:
                    mismatch.append((i - 1, curr_og, curr_re, curr_og_price, curr_re_price ))
                if re_diff != 30:
                    nonconti.append((i - 1, i, next_re, curr_re))
                    
        print(np.array(mismatch))      
        print(np.array(nonconti))
        
    def calculate_raw_power(self, market_dispatch):
        if market_dispatch < 0:
            raw_power = market_dispatch * 2
        else:
            raw_power = market_dispatch * 2 / self.discharge_efficiency
            
        return raw_power
    
    def calculate_revenue(self, price, market_dispatch):
        if market_dispatch < 0:
            revenue = price * market_dispatch / 0.991
        else:
            revenue = price * market_dispatch * 0.991
            
        return revenue
    
    def check_flag(self, raw_power, opening_capacity):
        max_charge = -min(self.battery_power, (self.battery_capacity - opening_capacity) / self.charge_efficiency * 2)
        max_discharge = min(self.battery_power, opening_capacity * 2)
        
        if raw_power < max_charge:
            return 1
        else:
            return 0
        if raw_power > max_discharge:
            return 2
        else:
            return 0
        
    def validation(self):
        df = self.re_df.copy()
        
        raw_power = []
        revenue = []
        flags = [] # For printing to final documents
        
        self.flagged = []
        
        for i in range(len(df)):
            md = df.loc[i, "Market Dispatch (MWh)"]
            price = df.loc[i, self.price_name]
            opening = df.loc[i, "Opening Capacity (MWh)"]
            
            raw_power.append(self.calculate_raw_power(md))
            revenue.append(self.calculate_revenue(price, md))
            
            flagging = self.check_flag(raw_power[i], opening)
            
            if flagging != 0:
                if flagging == 1:
                    string = 'Charging Limit Exceeded'
                elif flagging == 2:
                    string = 'Discharging Limit Exceeded'
                self.flagged.append((i, string))
                
            flags.append(flagging)
        
        df["Raw Power (MW)"] = pd.Series(raw_power)
        df["Market Revenue ($)"] = pd.Series(revenue)
        df["Flags (0/1/2)"] = pd.Series(flags)
        
        self.re_df = df
    
    def check(self):
        if len(self.flagged) == 0:
            print("Submitted File is Valid!")
        else:
            print("Submitted File Max Charge and Discharge Exceeded!")
            
    def save(self, filepath, filetype = "xlsx"):
        self.re_df['Time (UTC+10)'] = pd.to_datetime(self.re_df['Time (UTC+10)'])
        self.re_df["Year"] = pd.DatetimeIndex(self.re_df['Time (UTC+10)']).year
        
        if (filetype == "xlsx"):
            self.re_df.to_excel(filepath)
        elif (filetype == "csv"):
            self.re_df.to_csv(filepath)
            
    def __repr__(self):
        #if self.train_period == None and self.cv_period == None and self.test_period == None:
        revenue = self.re_df["Market Revenue ($)"].sum()
        return f"Total Revenue : {revenue}"

In [138]:
original = pd.read_excel("../../raw_data/market_data copy.xlsx")
example = pd.read_excel("../../preprocessed_data/First Algorithm/example.xlsx")
example_1 = pd.read_excel("../../preprocessed_data/First Algorithm/Victoria_data.xlsx")

In [166]:
submit = ValidityCheck(original, example)
submit
#submit.save("../../preprocessed_data/First Algorithm/output.xlsx", "xlsx")

[[52991 Timestamp('2021-01-09 00:00:00') Timestamp('2021-01-13 00:00:00')
  36.45 39.85]
 [52992 Timestamp('2021-01-09 00:30:00') Timestamp('2021-01-13 00:30:00')
  34.57 39.91]
 [52993 Timestamp('2021-01-09 01:00:00') Timestamp('2021-01-13 01:00:00')
  34.67 38.98]
 ...
 [63452 Timestamp('2021-08-14 22:30:00') Timestamp('2021-12-08 22:00:00')
  62.86 71.77]
 [63453 Timestamp('2021-08-14 23:00:00') Timestamp('2021-12-08 22:30:00')
  32.26 100.04]
 [63454 Timestamp('2021-08-14 23:30:00') Timestamp('2021-12-08 23:00:00')
  25.1 81.22]]
[[61919 61920 Timestamp('2021-09-01 00:00:00')
  Timestamp('2021-08-15 00:00:00')]]


Total Revenue : 127757268.62463573